# Solving a 9x9 grid cell Sudoku with the latest D-Wave Quantum Annealer via Cloud (D-Wave leap)

In [10]:
# for classical solver (simulated annealing)
import dimod
import operator
import dwavebinarycsp
import numpy as np
import math

In [11]:
from quantum_annealing_sudoku import quantum_annealing_sudoku
from quantum_annealing_sudoku.label_encoder import encode_var_labels, decode_var_labels

In [12]:
sudoku_9x9 = quantum_annealing_sudoku.QuantumAnnealingSudoku(grid_9x9 = True)


#get some function shortcuts for better readability 
check_sudoku = sudoku_9x9.check_sudoku
encode_board_to_binary = sudoku_9x9.encode_board_to_binary
decode_board_from_binary = sudoku_9x9.decode_board_from_binary
print_board = sudoku_9x9.print_board

In [13]:
board = ((0, 0, 3, 0, 2, 0, 6, 0, 0),
         (9, 0, 0, 3, 0, 5, 0, 0, 1),
         (0, 0, 1, 8, 0, 6, 4, 0, 0),
         (0, 0, 8, 1, 0, 2, 9, 0, 0),
         (7, 0, 0, 0, 0, 0, 0, 0, 8),
         (0, 0, 6, 7, 0, 8, 2, 0, 0),
         (0, 0, 2, 6, 0, 9, 5, 0, 0),
         (8, 0, 0, 2, 0, 3, 0, 0, 9),
         (0, 0, 5, 0, 1, 0, 3, 0, 0))
print_board(board)

 -  -  3 | -  2  - | 6  -  - 
 9  -  - | 3  -  5 | -  -  1 
 -  -  1 | 8  -  6 | 4  -  - 
---------|---------|---------
 -  -  8 | 1  -  2 | 9  -  - 
 7  -  - | -  -  - | -  -  8 
 -  -  6 | 7  -  8 | 2  -  - 
---------|---------|---------
 -  -  2 | 6  -  9 | 5  -  - 
 8  -  - | 2  -  3 | -  -  9 
 -  -  5 | -  1  - | 3  -  - 



In [14]:
board = ((0, 0, 0, 8, 9, 0, 0, 0, 0),
         (2, 6, 0, 0, 0, 4, 0, 0, 0),
         (0, 0, 4, 0, 0, 0, 0, 0, 0),
         (0, 0, 5, 3, 0, 0, 4, 0, 9),
         (0, 8, 0, 0, 0, 0, 7, 5, 0),
         (6, 0, 0, 0, 0, 0, 0, 0, 3),
         (0, 7, 0, 0, 2, 6, 0, 0, 4),
         (1, 2, 0, 5, 0, 3, 0, 0, 0),
         (0, 0, 6, 0, 0, 0, 2, 0, 0))
print_board(board)

 -  -  - | 8  9  - | -  -  - 
 2  6  - | -  -  4 | -  -  - 
 -  -  4 | -  -  - | -  -  - 
---------|---------|---------
 -  -  5 | 3  -  - | 4  -  9 
 -  8  - | -  -  - | 7  5  - 
 6  -  - | -  -  - | -  -  3 
---------|---------|---------
 -  7  - | -  2  6 | -  -  4 
 1  2  - | 5  -  3 | -  -  - 
 -  -  6 | -  -  - | 2  -  - 



In [15]:
correct_board = ((5, 3, 7, 8, 9, 1, 6, 4, 2),
         (2, 6, 1, 7, 5, 4, 3, 9, 8),
         (8, 9, 4, 6, 3, 2, 1, 7, 5),
         (7, 1, 5, 3, 6, 8, 4, 2, 9),
         (3, 8, 2, 4, 1, 9, 7, 5, 6),
         (6, 4, 9, 2, 7, 5, 8, 1, 3),
         (9, 7, 3, 1, 2, 6, 5, 8, 4),
         (1, 2, 8, 5, 4, 3, 9, 6, 7),
         (4, 5, 6, 9, 8, 7, 2, 3, 1))
print_board(correct_board)

 5  3  7 | 8  9  1 | 6  4  2 
 2  6  1 | 7  5  4 | 3  9  8 
 8  9  4 | 6  3  2 | 1  7  5 
---------|---------|---------
 7  1  5 | 3  6  8 | 4  2  9 
 3  8  2 | 4  1  9 | 7  5  6 
 6  4  9 | 2  7  5 | 8  1  3 
---------|---------|---------
 9  7  3 | 1  2  6 | 5  8  4 
 1  2  8 | 5  4  3 | 9  6  7 
 4  5  6 | 9  8  7 | 2  3  1 



In [16]:
check_sudoku(board)

11

In [17]:
check_sudoku(correct_board)

0

In [18]:
constant = 0
N = 9
cell_qubo = {}
linear = {}
quad = {}
penalty_weight = 1
for i in range(1,N+1):
        for j in range(1,N+1):
            for k1 in range(1,N+1):  
                    var_1 = encode_var_labels(i,j,k1)
                    for k2 in range(1,N+1):
                            var_2 = encode_var_labels(i,j,k2)
                            if var_1 == var_2:
                                linear[var_1] = -1*penalty_weight
                            else:
                                quad[var_1,var_2]= 2*penalty_weight
                        #linear[var_1] = 2
                    
            constant+=1
            

cell_qubo[()] = constant*penalty_weight
cell_qubo["linear"] = linear
cell_qubo["quadratic"] = quad

#bqm = dimod.BinaryQuadraticModel(cell_qubo,constant,dimod.Vartype.BINARY)

In [19]:
penalty_weight=1
constant = 0
N = 9
column_qubo = {}
lin_column={}
quad_column={}
binary_board = encode_board_to_binary(board)
for k in range(1,N):
        for j in range(1,N):
            for i1 in range(1,N):
                var_1 = encode_var_labels(i1,j,k)
                for i2 in range(1,N):               
                    var_2 = encode_var_labels(i2,j,k)
                    if var_1 == var_2:
                        lin_column[var_1] = -1*penalty_weight
                    else:
                        quad_column[var_1,var_2] = 2*penalty_weight
        constant+=1
        
column_qubo[()] = constant*penalty_weight
column_qubo['linear'] = lin_column
column_qubo['quadratic'] = quad_column

In [20]:
constant = 0
N = 9
row_qubo = {}
linear_row ={}
quadratic_row={}
penalty_weight=2
binary_board = encode_board_to_binary(board)
for k in range(1,N):
        for i in range(1,N):
            for j1 in range(1,N):
                var_1 = encode_var_labels(i,j1,k)
                for j2 in range(1,N):      
                    var_2 = encode_var_labels(i,j2,k)
                    if var_1 == var_2:
                        linear_row[var_1] = -1*penalty_weight
                    else:
                        quadratic_row[var_1, var_2] = 2*penalty_weight
        constant+=1
        
row_qubo[()] = constant*penalty_weight
row_qubo['linear'] = linear_row
row_qubo['quadratic'] = quadratic_row

In [21]:
binary_board = encode_board_to_binary(board)
hint = []
penalty_weight = 4

for k, color in enumerate(binary_board):
    for i, row in enumerate(color):
        for j, cell in enumerate(row):
            if cell>0:
                hint.append([i+1,j+1,k+1])
                
hint_qubo = {}
constant=0
for (i, j, k) in hint:
    re_label = encode_var_labels(i,j,k) 
    hint_qubo[re_label] = -1*penalty_weight
    constant += 1

hint_qubo[()] = constant*penalty_weight


In [22]:
from math import sqrt
constant = 0
N = 9
duplicate_qubo = {}
dupli_linear = {}
dupli_quadr = {}
sqrtN = int(sqrt(N))
for grid_i in range(sqrtN):
    for grid_j in range(sqrtN):
        for k in range(N):
            # there can be only one k in the same subgrid.
            for i1 in range(grid_i * 3, grid_i * 3 + 3):
                for j1 in range(grid_j * 3, grid_j * 3 + 3):
                    for i2 in range(grid_i * 3, grid_i * 3 + 3):
                        var_1 = encode_var_labels(i1+1,j1+1,k+1)
                        for j2 in range(grid_j * 3, grid_j * 3 + 3):
                            var_2 = encode_var_labels(i2+1,j2+1,k+1)
                            if var_1 == var_2:
                                dupli_linear[var_1] = -1
                            else:
                                dupli_quadr[var_1,var_2] = 1
            constant+=1

            
duplicate_qubo[()] = constant
duplicate_qubo['linear'] = dupli_linear
duplicate_qubo['quadratic'] = dupli_quadr

In [23]:
def append_linear(result_qubo_linear, qubo):    
    if not qubo.get('linear'):
        return result_qubo_linear
    for index, value in qubo['linear'].items():
        if result_qubo_linear.get(index):
            result_qubo_linear[index] += value     
        else:
            result_qubo_linear[index] = value
            
    return result_qubo_linear
            
def append_quadratic(result_qubo_quadratic, qubo):    
    if not qubo.get('quadratic'):
        return result_qubo_quadratic
    for index, value in qubo['quadratic'].items():
        if result_qubo_quadratic.get(index):
            result_qubo_quadratic[index] += value 
        else:
            result_qubo_quadratic[index] = value
            
    return result_qubo_quadratic
            

In [24]:
# qubos that are needed to achieve the constraints
qubos=[cell_qubo, column_qubo, row_qubo, hint_qubo, duplicate_qubo]

In [26]:
result_qubo_linear = {}
result_qubo_quadratic = {}
for qubo in qubos:    
    if qubo is not None:
        result_qubo_linear = append_linear(result_qubo_linear, qubo)
        result_qubo_quadratic = append_quadratic(result_qubo_quadratic, qubo)

In [27]:
constant/2

40.5

In [28]:
constant= cell_qubo.get((),0) + hint_qubo.get((),0) + row_qubo.get((),0) + column_qubo.get((),0) + duplicate_qubo.get((),0)
bqm = dimod.BinaryQuadraticModel(result_qubo_linear,result_qubo_quadratic, constant,dimod.Vartype.BINARY)

In [29]:
# test run
import neal
sampler = neal.SimulatedAnnealingSampler()
sample_set=sampler.sample(bqm, num_reads=30,num_sweeps=2222)


In [30]:
from random import randrange
import time

def find_optimal_solution(best_solution_global, current_solution, current_energy):
    best_solution = {}
    error_count=20
    iteration = 0
    sampler = neal.SimulatedAnnealingSampler()
    start_time = time.perf_counter()
    while error_count>0:
        """
        sample_set = sampler.sample(bqm, seed=1234, beta_range=[0.1, 4.2],
                                        num_reads=10, num_sweeps=20000,
                                       beta_schedule_type='geometric')
        """
        random_seed = randrange(9000)
        #random_seed = 5832
        num_reads = randrange(12000)
        num_sweeps = randrange(60000)
        num_reads = 5
        num_sweeps=50
        #num_sweeps*=iteration
        pre_anneal = time.perf_counter()
        sample_set=sampler.sample(bqm,seed=random_seed ,num_reads=num_reads,num_sweeps=num_sweeps)
        iteration+=1
        post_anneal = time.perf_counter()
        
        annealing_time = post_anneal-pre_anneal
        for solution, energy in sample_set.data(['sample', 'energy']):
            binary_solution_board= np.zeros((9, 9, 9))
            for index, value in solution.items():
                if type(index) is int and index>0:
                    board_index = decode_var_labels(index)
                    binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
            solution_board=decode_board_from_binary(binary_solution_board)
            error_count_temp = check_sudoku(solution_board)
            
            
            
            current_solution.append(1)
            current_energy.append(energy)
            current_solution[:] = []
            current_energy[:] = []
            current_solution.append(solution)
            current_energy.append(energy)
            
            
        overall_time = time.perf_counter()-start_time
        if error_count_temp<error_count:
            best_solution = solution_board
            best_solution_global.append(solution)
            error_count=error_count_temp
            print("\nError Count:",error_count, "iteration:", iteration, "Energy:", energy,
                  "Seed:", random_seed, "num_reads:", num_reads, "sweeps:", num_sweeps, 
                 "Annealing Time:", annealing_time, "Overall Time:", overall_time)
            
        if iteration%10 == 0:
            print("\nCurrent State:",
                  "Error Count:",error_count_temp, "iteration:", iteration, "Energy:", energy,
                 "Seed:", random_seed, "num_reads:", num_reads, "sweeps:", num_sweeps,
                 "Annealing Time:", annealing_time, "Overall Time:", overall_time)
            

        

In [31]:
import multiprocessing
manager = multiprocessing.Manager()
best_solution = manager.list()
current_solution = manager.list()
current_energy = manager.list()


process = multiprocessing.Process(target=find_optimal_solution, 
            
                                  args= (best_solution, current_solution, current_energy))
process.start()




Error Count: 6 iteration: 1 Energy: -23.0 Seed: 2310 num_reads: 5 sweeps: 50 Annealing Time: 0.1149751000048127 Overall Time: 0.28603471600217745

Error Count: 5 iteration: 2 Energy: -38.0 Seed: 8593 num_reads: 5 sweeps: 50 Annealing Time: 0.1113939690112602 Overall Time: 0.6215770370035898

Error Count: 4 iteration: 3 Energy: -35.0 Seed: 4781 num_reads: 5 sweeps: 50 Annealing Time: 0.12242604797938839 Overall Time: 0.9851448560075369

Current State: Error Count: 7 iteration: 10 Energy: -30.0 Seed: 3291 num_reads: 5 sweeps: 50 Annealing Time: 0.159658265998587 Overall Time: 3.1861984099959955

Error Count: 3 iteration: 14 Energy: -36.0 Seed: 5466 num_reads: 5 sweeps: 50 Annealing Time: 0.10217202198691666 Overall Time: 4.388904931984143

Current State: Error Count: 6 iteration: 20 Energy: -25.0 Seed: 3157 num_reads: 5 sweeps: 50 Annealing Time: 0.10379416300565936 Overall Time: 6.177373496000655

Current State: Error Count: 4 iteration: 30 Energy: -37.0 Seed: 1297 num_reads: 5 sweeps:

In [32]:
import time
time.sleep(10)
#wait for terminating in order to have  
#results when doing a (quick) autorun

process.terminate()
process

<Process name='Process-2' pid=1705860 parent=1633413 started>

In [33]:
#sample_set = list(current_solution)
sample_set = list(best_solution)

In [34]:
solution={}
solution_2={}
solution_3={}
for number, solution in enumerate(sample_set):
    if number==0:
        solution = solution
    elif number==1:
        solution_2 = solution
    elif number==2:
        solution_3 = solution
        
    
last_solution = solution

In [35]:
binary_solution_board= np.zeros((9, 9, 9))
for index, value in solution.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value


In [36]:
solution_board=decode_board_from_binary(binary_solution_board)
solution_board

array([[8., 3., 6., 5., 1., 9., 9., 6., 7.],
       [5., 1., 9., 3., 6., 7., 4., 2., 8.],
       [7., 5., 6., 4., 8., 2., 1., 3., 5.],
       [3., 7., 9., 2., 5., 1., 6., 8., 1.],
       [2., 8., 1., 7., 4., 6., 3., 5., 4.],
       [6., 4., 5., 9., 3., 8., 2., 9., 7.],
       [9., 6., 7., 4., 2., 3., 5., 1., 8.],
       [4., 2., 8., 6., 9., 5., 9., 7., 6.],
       [3., 5., 1., 8., 1., 7., 4., 3., 2.]])

In [37]:
print_board(board)

 -  -  - | 8  9  - | -  -  - 
 2  6  - | -  -  4 | -  -  - 
 -  -  4 | -  -  - | -  -  - 
---------|---------|---------
 -  -  5 | 3  -  - | 4  -  9 
 -  8  - | -  -  - | 7  5  - 
 6  -  - | -  -  - | -  -  3 
---------|---------|---------
 -  7  - | -  2  6 | -  -  4 
 1  2  - | 5  -  3 | -  -  - 
 -  -  6 | -  -  - | 2  -  - 



In [38]:
print_board(solution_board)
print("Error Count:",check_sudoku(solution_board))
print("BQM Energy:", bqm.energy(solution))

 8  3  6 | 5  1  9 | 9  6  7 
 5  1  9 | 3  6  7 | 4  2  8 
 7  5  6 | 4  8  2 | 1  3  5 
---------|---------|---------
 3  7  9 | 2  5  1 | 6  8  1 
 2  8  1 | 7  4  6 | 3  5  4 
 6  4  5 | 9  3  8 | 2  9  7 
---------|---------|---------
 9  6  7 | 4  2  3 | 5  1  8 
 4  2  8 | 6  9  5 | 9  7  6 
 3  5  1 | 8  1  7 | 4  3  2 

Error Count: 3
BQM Energy: -36.0


In [39]:
binary_solution_board= np.zeros((9, 9, 9))
for index, value in solution_2.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
print_board(solution_board)
print("Error Count:",check_sudoku(solution_board))

 8  3  6 | 5  1  9 | 9  6  7 
 5  1  9 | 3  6  7 | 4  2  8 
 7  5  6 | 4  8  2 | 1  3  5 
---------|---------|---------
 3  7  9 | 2  5  1 | 6  8  1 
 2  8  1 | 7  4  6 | 3  5  4 
 6  4  5 | 9  3  8 | 2  9  7 
---------|---------|---------
 9  6  7 | 4  2  3 | 5  1  8 
 4  2  8 | 6  9  5 | 9  7  6 
 3  5  1 | 8  1  7 | 4  3  2 

Error Count: 3


In [40]:
binary_solution_board= np.zeros((9, 9, 9))
for index, value in solution_3.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
print_board(solution_board)
print("Error Count:",check_sudoku(solution_board))

 8  3  6 | 5  1  9 | 9  6  7 
 5  1  9 | 3  6  7 | 4  2  8 
 7  5  6 | 4  8  2 | 1  3  5 
---------|---------|---------
 3  7  9 | 2  5  1 | 6  8  1 
 2  8  1 | 7  4  6 | 3  5  4 
 6  4  5 | 9  3  8 | 2  9  7 
---------|---------|---------
 9  6  7 | 4  2  3 | 5  1  8 
 4  2  8 | 6  9  5 | 9  7  6 
 3  5  1 | 8  1  7 | 4  3  2 

Error Count: 3


In [41]:
binary_solution_board= np.zeros((9, 9, 9))
for index, value in last_solution.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
print_board(solution_board)
print("Error Count:",check_sudoku(solution_board))

 8  3  6 | 5  1  9 | 9  6  7 
 5  1  9 | 3  6  7 | 4  2  8 
 7  5  6 | 4  8  2 | 1  3  5 
---------|---------|---------
 3  7  9 | 2  5  1 | 6  8  1 
 2  8  1 | 7  4  6 | 3  5  4 
 6  4  5 | 9  3  8 | 2  9  7 
---------|---------|---------
 9  6  7 | 4  2  3 | 5  1  8 
 4  2  8 | 6  9  5 | 9  7  6 
 3  5  1 | 8  1  7 | 4  3  2 

Error Count: 3


In [42]:
bqm.energy(solution)

-36.0

In [44]:
from dwave.system import DWaveSampler, EmbeddingComposite
# Use a D-Wave system as the sampler
sampler = DWaveSampler() 

print("QPU {} was selected.".format(sampler.solver.name))

QPU Advantage_system1.1 was selected.


In [45]:
# Set up a D-Wave system as the sampler
sampler = EmbeddingComposite(sampler)


In [46]:
bqm_graph = bqm.to_networkx_graph()

In [49]:
print('Number of problem nodes (variables):',len(bqm_graph.nodes))
print('Number of problem edges (couplings):',len(bqm_graph.edges))

Number of problem nodes (variables): 729
Number of problem edges (couplings): 8520


### Unfortunately, as of this point in time (2021) the current D-Wave system (D-Wave QA one) with pegasus architecture is probably too small to be able to find an embedding for a full 9x9 Sudoku problem.

At least minorminor.find_embedding, which works probabilisticly, does not find a valid assignment (in a feasible time-frame).

In [64]:
from dimod import BinaryQuadraticModel
from dwave.embedding import embed_bqm, unembed_sampleset
from dwave.system.samplers import DWaveSampler
from minorminer import find_embedding
from dwave.embedding.chain_breaks import majority_vote

solver = DWaveSampler()

__, target_edgelist, target_adjacency = solver.structure

#bqm = BinaryQuadraticModel.from_qubo(Q)

emb = find_embedding(bqm.to_qubo()[0], target_edgelist)

#embedded_bqm = embed_bqm(bqm, emb, target_adjacency)

#result = solver.sample(embedded_bqm, num_reads=1)

#unembedded = unembed_sampleset(result, emb, bqm, chain_break_method=majority_vote(bqm, emb))

KeyboardInterrupt: embedding cancelled by keyboard interrupt

In [50]:
# another way to maybe find an embedding for the pegasus architecture  @some point

# don't know how to choose a proper value for m yet..

from dwave.embedding.pegasus import find_clique_embedding

find_clique_embedding(k=bqm.to_networkx_graph(), m=100)



KeyboardInterrupt: 

In [103]:
len(target_adjacency)

5436

In [72]:
embedded_bqm = embed_bqm(source_bqm=bqm, target_adjacency=target_adjacency)

AttributeError: 'NoneType' object has no attribute 'items'

In [66]:
embedded_bqm = embed_bqm(bqm, emb, target_adjacency)

NameError: name 'emb' is not defined

In [ ]:
result = solver.sample(embedded_bqm, num_reads=1)

In [ ]:
unembedded = unembed_sampleset(result, emb, bqm, chain_break_method=majority_vote(bqm, emb))

In [ ]:
import dwave.inspector
dwave.inspector.show(sampleset)   